# Описание проекта


Оператор мобильной связи «Мегалайн» выяснил: многие клиенты пользуются архивными тарифами.

Они хотят построить систему, способную проанализировать поведение клиентов и предложить пользователям новый тариф: «Смарт» или «Ультра».

В нашем распоряжении данные о поведении клиентов, которые уже перешли на эти тарифы (из проекта курса «Статистический анализ данных»). Нужно построить модель для задачи классификации, которая выберет подходящий тариф. Предобработка данных не понадобится — мы её уже сделали.

Нужно построить  модель с максимально большим значением accurac
Чтобы сдать проект успешно, нужно довести долю правильных ответов по крайней мере до 0.75.

# Инструкция по выполнению проекта


- Открыть файл с данными и изучите его.

- Разделить исходные данные на обучающую, валидационную и тестовую выборки.

- Исследовать качество разных моделей, меняя гиперпараметры. Написать краткие  выводы исследования.

- Проверить качество модели на тестовой выборке.

- Дополнительное задание: проверить модели на вменяемость.

# Описание данных


Каждый объект в наборе данных — это информация о поведении одного пользователя за месяц.
Известно:


- `сalls` — количество звонков,
- `minutes` — суммарная длительность звонков в минутах,
- `messages` — количество sms-сообщений,
- `mb_used` — израсходованный интернет-трафик в Мб,
- `is_ultra` — каким тарифом пользовался в течение месяца («Ультра» — 1, «Смарт» — 0).

## Откроем файл и изучим его

In [ ]:
#Импортируем необходимые библиотеки

import pandas as pd
from sklearn.tree import DecisionTreeClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score
from sklearn.model_selection import train_test_split
from sklearn.dummy import DummyClassifier

In [ ]:
# Откроем файл
try:
    df = pd.read_csv('/datasets/users_behavior.csv')
except:
    df = pd.read_csv('C:\\Users\\magas\\Desktop\\Проекты\\users_behavior.csv')

In [ ]:
# Посмотрим информацию

df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 3214 entries, 0 to 3213
Data columns (total 5 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   calls     3214 non-null   float64
 1   minutes   3214 non-null   float64
 2   messages  3214 non-null   float64
 3   mb_used   3214 non-null   float64
 4   is_ultra  3214 non-null   int64  
dtypes: float64(4), int64(1)
memory usage: 125.7 KB


In [ ]:
# Перепроверим пропуски

df.isna().sum()

calls       0
minutes     0
messages    0
mb_used     0
is_ultra    0
dtype: int64

In [ ]:
# и дубликаты
print(df.duplicated().sum())

0


In [ ]:
df['is_ultra'].value_counts()

0    2229
1     985
Name: is_ultra, dtype: int64

### Промежуточный вывод

Данные, действительно, в отличном состоянии. Предобработка не требуется.

В таблице 3214 строк, каждая из которых характеризует поведение одного пользователя в течении месяца.
Данных достаточно для обучения модели.


## Разделим исходные данные на обучающую, валидационную и тестовую выборки


Так как `train_test_split` делит общую выборку только на две части, сначала выделим тестовую часть, затем из оставшейся выделим валидационную.

Тестовая и валидационная выборки должны быть равны и меньше обучающей, потому разделение будет 3/1/1(20% / 20% / 60%)


In [ ]:
# Выделим тестовую часть

df_train_valid, df_test = train_test_split(df, test_size=0.20, random_state=12345)

In [ ]:
df_train, df_valid = train_test_split(df_train_valid, test_size=0.25, random_state=12345)

In [ ]:
# Проверим размеры выборок

print(df_test.shape[0])
print(df_valid.shape[0])
print(df_train.shape[0])

643
643
1928


### Промежуточный вывод

Данный разделили, теперь можно приступать к обучению

## Исследовать качество разных моделей, меняя гиперпараметры. Написать краткие выводы исследования.

**В первую очередь, разделим обучаюдие признаки и целевой**


In [ ]:
# Обучающие

features_train = df_train.drop(['is_ultra'],axis=1)
target_train = df_train['is_ultra']

In [ ]:
# Валидационные

features_valid = df_valid.drop(['is_ultra'],axis=1)
target_valid = df_valid['is_ultra']

In [ ]:
# Тестовые

features_test = df_test.drop(['is_ultra'],axis=1)
target_test = df_test['is_ultra']

Теперь приступим к обучению.

### Сперва возьмем дерево решений, перебрав циклом глубину от 1 до 10 и выбрав наилучший

In [ ]:
# Создадим переменные, в которые сохраним лучшую модель и лучший результат accuracy
best_tree_model = None
best_result = 0
best_depth  = 0

# теперь пройдемся циклом, подставляя значения максимальной глубины

for depth in range(1,11):

    best_tree_model = DecisionTreeClassifier(random_state=12345, max_depth=depth)
    best_tree_model.fit(features_train,target_train)
    prediction_valid = best_tree_model.predict(features_valid)
    result = accuracy_score(target_valid,prediction_valid)
    # Можно узнать результат accuracy и просто методом score.
    # result = best_tree_model.score(features_valid,target_valid)

# Найдем лучшие результаты

    if result > best_result:
        best_depth = depth
        best_result = result
        best_tree_model = best_tree_model

# Выведем на экран

print('Наилучший результат:', best_result,)
print('Максимальная глубина:', best_depth)

Наилучший результат: 0.7744945567651633
Максимальная глубина: 7


### Теперь случайный лес

**Так же с помощью циклом подберем наилучшее количество деревьев и глубину каждого**

In [ ]:
# Создадим переменные, в которые сохраним лучшую модель и лучший результат accuracy
best_forest_model = None
best_result = 0
best_depth  = 0
best_est = 0
# теперь пройдемся циклом, подставляя значения максимальной глубины

for est in range(1,51):
    for depth in range(1,11):

        best_forest_model = RandomForestClassifier(random_state=12345, max_depth=depth, n_estimators=est)
        best_forest_model.fit(features_train,target_train)
        prediction_valid = best_forest_model.predict(features_valid)
        result = accuracy_score(target_valid,prediction_valid)
        # Можно узнать результат accuracy и просто методом score.
        # result = best_forest_model.score(features_valid,target_valid)

# Найдем лучшие результаты

        if result > best_result:
            best_depth = depth
            best_result = result
            best_forest_model = best_forest_model
            best_est = est

# Выведем на экран

print('Наилучший результат:', best_result)
print('Максимальная глубина:', best_depth)
print('Количество деревьев:', best_est)

Наилучший результат: 0.7993779160186625
Максимальная глубина: 10
Количество деревьев: 36


### Логистическая регрессия

In [ ]:
linear_model = LogisticRegression(random_state=12345, solver='lbfgs', max_iter=100)
linear_model.fit(features_train,target_train)
prediction_valid = linear_model.predict(features_valid)

result = accuracy_score(target_valid,prediction_valid)

# Можно узнать результат accuracy и просто методом score.
# result = best_forest_model.score(features_valid,target_valid)

print('Наилучший результат:', result)

Наилучший результат: 0.7262830482115086


### Промежуточный вывод

Проверили три алгоритма, с помощью циклов подобрав оптимальные глубины и количество деревьев.

Так как перед нами стоит задача выбрать самое высокое качестве, не обращая внимание на скорость, лучше всего на валидационной выборке себя показал Случайный лес с результатом в 0.79937791601866255, при количесвте деревьей -20 , глубине каждого дерева - 10.

Дерево решений - результат 0.7744945567651633, при глубине в 7

Логистическая регрессия показала худший результат из трех - 0.7262830482115086

## Проверить качество модели на тестовой выборке.

Модели обучены, теперь проверим, как они будут предсказывать на тестовой выборке

In [ ]:
# Дерево решений

pridiction_test = best_tree_model.predict(features_test)
accuracy_score(target_test,pridiction_test)

0.7713841368584758

In [ ]:
# Случайный лес

pridiction_test = best_forest_model.predict(features_test)
accuracy_score(target_test,pridiction_test)

0.7993779160186625

In [ ]:
# Логистическая регрессия

pridiction_test = linear_model.predict(features_test)
accuracy_score(target_test,pridiction_test)

0.7589424572317263

### Промежуточный вывод

Проверили модели на тестовых выборках и результаты такие же, как и при работе с валидационной выборкой:

Лучше всего справился случайный лес, почти один в один повторив результат проверки на валидационной выборке - 0.7993779160186625

Далее дерево решений, практически 1 в 1 повторившее результат - 0.7713841368584758  

Замыкает тройку логистическая регрессия, показашая на тестовой выборке лучше результат, чем  на валидационной -

Так как перед нами стоит задача выбрать самое высокое качестве, не обращая внимание на скорость, выбор падает на **случайный лес**

## Дополнительное задание: проверить модели на вменяемость.

Для проверки вменяемости модели воспользуемся "моделью-идиоткой" - `DummyClassifier`, обучим ее на тех же признаках, что и основные модели и сравним результаты

In [ ]:
dummy_model = DummyClassifier(strategy='most_frequent')

dummy_model.fit(features_train,target_train)

result = dummy_model.score(features_test,target_test)
print(result)

0.6951788491446346


### Промeжуточный вывод

Как видим, результат ниже, чем у основных моделей, потому можем считать модели, обученные в проекте, вменяемыми.

# ОБЩИЙ ВЫВОД

В данном проекте мы проанализировали данные о поведении пользователей для создания модели, которая помогла бы корректно предлагать клиента подходящий им тариф.

Для этого мы проведали следующие шаги:

- Открыли файл с данными и изучили его: посмотрели общую информацию. перепроверили на пропуски и дубликаты

- Разделили исходные данные на обучающую, валидационную и тестовую выборки в соотношении 3/1/1. В итогу у нас получились выборки размерами 1928, 643, 643 соответственно

- Создали 3 модели под три алгоритма обучения:
    `best_tree_model` - решающее дерево
    `best_forest_model` - случайный лес
    `leaner_model`- Логистическая регрессия.
    
- Исследовали и сравнили  качество разных моделей на ваидационной выборке, с помощью циклов меняя и подбирая лучшие гиперпараметры

- Проверить качество модели на тестовой выборке и сравнили результаты

- Дополнительно проверили модели на вменяемость с помощью `DummyClassifier`.


По результатам всех проверок, самой точной моделью оказалась, обученная на "случайном лесе", `best_forest_model` с результатом `0.7993779160186625`, c параметрами: количество деревьев - 36, максимальная глубина - 10

Следом идет `best_tree_model`, обученная на "деревее решений" - `0.7713841368584758`, максимальная глубина - 7

Последняя модель - `leaner_model` - 0.7589424572317263

Так же все три модели показали вменяемость при сравнении с "моделью-идиоткой" - `dummy_model` - результат каждой оказался выше результата этой модели